In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.compose import ColumnTransformer

In [2]:
from form_suppliers_df import form_suppliers_df, filter_suppliers
from preprocessor import preprocess_tender_data

Сначала определим создание пула поставщиков по обучающим данным (я возьму до 2023 года), а новый тендер будем брать из теста по 2024 и 2025 году, то есть как и было сделано ранее (потому что до этого заполняла пропуски в 2024 и 2025 через ближайших соседей, хотя потенциально можно тут использовать и фулл исторические данные, пока не знаю)

In [3]:
data = pd.read_excel('data_final_res.xlsx')

In [4]:
data = preprocess_tender_data(data)

Удалено строк после фильтрации невалидных тендеров: 2
  Удалено строк с отсутствующей Сферой деятельности: 14
Заполнено пропусков в 'Стоимость(руб.) Заказчик': 0 (должно быть 0)
Размер 'train_data' для расчета статистики: 100564 строк
  Используется фиксированная дата для recent_activity_ratio: 2025-04-10
Колонки для KNNImputer: ['win_rate', 'region_wins', 'region_win_rate', 'customer_wins', 'customer_win_rate', 'sphere_wins', 'sphere_win_rate', 'total_wins', 'avg_price_drop', 'recent_activity_ratio', 'competitors_per_tender', 'avg_competitors_in_region', 'avg_competitors_in_sphere', 'avg_competitors_in_customer']


In [5]:
supplier_historical_data = form_suppliers_df(data)

Создано 41047 профилей с историческими данными поставщиков.


Найдем тендеры для тестирования

In [6]:
from form_suppliers_df import select_test_tenders

train_data = data[data['year'].isin(range(2019, 2024))] 
valid_test = select_test_tenders(train_data)

41047 уникальных поставщиков в данных

Найдено 13436 валидных тендеров.

Тендер 1/5:
ID: 1501657
Количество участников: 4
Информация об участниках:
- Участник -
ИНН: 5406593872
Участий в данных: 1
Побед в данных:   0
----------------------------------------
* ПОБЕДИТЕЛЬ *
ИНН: 4205357009
Участий в данных: 2
Побед в данных:   2
----------------------------------------
- Участник -
ИНН: 4217192055
Участий в данных: 1
Побед в данных:   0
----------------------------------------
- Участник -
ИНН: 4217177353
Участий в данных: 1
Побед в данных:   0
----------------------------------------

Тендер 2/5:
ID: 344200020620000000
Количество участников: 7
Информация об участниках:
- Участник -
ИНН: 4632254286
Участий в данных: 19
Побед в данных:   8
----------------------------------------
* ПОБЕДИТЕЛЬ *
ИНН: 4632057513
Участий в данных: 3
Побед в данных:   3
----------------------------------------
* ПОБЕДИТЕЛЬ *
ИНН: 4632176951
Участий в данных: 24
Побед в данных:   14
---------------------------

In [7]:
from form_suppliers_df import format_tender_info_dict

In [8]:
tender_subset_df = data[data['Реестровый номер публикации'] == 32110673060]
new_tender_info = format_tender_info_dict(tender_subset_df)

In [9]:
new_tender_info

{'Регион поставки': 'Иркутская область',
 'Город поставки': 'Иркутская область',
 'Сфера деятельности': '[ОКПД2 71.12] Услуги в области инженерно-технического проектирования и связанные технические консультативные услуги, [ОКВЭД2 71.12] Деятельность в области инженерных изысканий, инженерно-технического проектирования, управления проектами строительства, выполнения строительного контроля и авторского надзора, предоставление технических консультаций в этих областях',
 'Заказчик': 'ООО "БЭК-РЕМОНТ"',
 'ИНН заказчика': np.int64(3808271889),
 'Стоимость(руб.) Заказчик': np.float64(1268000.0),
 'Дата публикации': '2021-09-27 16:36:08',
 'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ': '2021-10-06 17:00:00',
 'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ': '2021-09-27 00:00:00',
 'Дата окончания проведения торгов': '2021-10-11 00:00:00',
 'Форма публикации': 'Торговая процедура',
 'Сни

In [10]:
new_tender_df = pd.DataFrame([new_tender_info])
use_filtered = True 

if use_filtered:
    if 'regions_set' in supplier_historical_data.columns and 'spheres_set' in supplier_historical_data.columns:
        active_supplier_pool = filter_suppliers(supplier_historical_data, new_tender_df)
    else:
        print("Предупреждение: Столбцы 'regions_set' или 'spheres_set' отсутствуют. Используются все поставщики.")
        active_supplier_pool = supplier_historical_data.copy() 
else:
    active_supplier_pool = supplier_historical_data.copy() 

num_suppliers_to_predict = active_supplier_pool.shape[0]

Получено 41 поставщиков по региону 'Иркутская область' и сфере '[ОКПД2 71.12] Услуги в области инженерно-технического проектирования и связанные технические консультативные услуги, [ОКВЭД2 71.12] Деятельность в области инженерных изысканий, инженерно-технического проектирования, управления проектами строительства, выполнения строительного контроля и авторского надзора, предоставление технических консультаций в этих областях'


In [11]:
if num_suppliers_to_predict > 0:
    tender_df_repeated = pd.concat([new_tender_df] * num_suppliers_to_predict, ignore_index=True)
    suppliers_reset = active_supplier_pool.reset_index(drop=True)
    prediction_input_df = pd.concat([suppliers_reset, tender_df_repeated], axis=1)

    num_features = [
        "avg_price_drop",
        "Стоимость(руб.) Заказчик", 
        "Снижение на торгах,%"
    ]
    cat_features = [
        "Сфера деятельности", 
        "Регион поставки",
        "Город поставки", 
        "Форма публикации", 
        "Тип торгов" 
    ]
    passthrough_features = [
        "Статус допуска", 
        "РНП ранее",
        "is_january", 
        "publication_peak_9_11", 
        "app_end_noon_12", 
        "app_start_night_12", 
        "trade_end_night_12", 
        "days_between", 
        'year',
        'month', 
        'win_rate', 
        'region_wins',
        'region_win_rate',
        'customer_wins',
        'customer_win_rate',
        'sphere_wins',
        'sphere_win_rate', 
        'total_wins', 
        'recent_activity_ratio', 
        'competitors_per_tender',
        'avg_competitors_in_region', 
        'avg_competitors_in_sphere', 
        'avg_competitors_in_customer' 
    ]

    all_features_needed = num_features + cat_features + passthrough_features
    final_columns = ['ИНН поставщика'] + all_features_needed
    missing_cols = [col for col in final_columns if col not in prediction_input_df.columns]
    if missing_cols:
        print(f"Следующие столбцы отсутствуют в prediction_input_df: {missing_cols}")
    else:
        prediction_input_final = prediction_input_df[final_columns].copy() 
        # Сохраняем ИНН поставщиков отдельно для сопоставления результатов
        supplier_inns_predict = prediction_input_final['ИНН поставщика']
        prediction_data_for_model = prediction_input_final.drop(columns=['ИНН поставщика'])

In [12]:
#prediction_data_for_model

Теперь можем протестировать модельки

***Пока не знаю, насколько корректное решение со Снижением на торгах и как его все таки правильно использовать***

In [13]:
import joblib

In [14]:
loaded_knn_pipeline = joblib.load('models_joblib/best_knn_pipeline_model.joblib')
predictions_knn = loaded_knn_pipeline.predict_proba(prediction_data_for_model)[:, 1]

In [15]:
loaded_svm_pipeline = joblib.load('models_joblib/best_svm_pipeline_model.joblib')
predictions_svm = loaded_svm_pipeline.predict_proba(prediction_data_for_model)[:, 1]

In [16]:
from tqdm.auto import tqdm 
from torch.utils.data import DataLoader 
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/Users/polina/b2b_reccomendation_systems/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
class TenderNet(nn.Module):
    def __init__(self, input_size: int, hidden_size1: int = 100, hidden_size2: int = 50, dropout_rate: float = 0.3):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.Tanh(),
            nn.BatchNorm1d(hidden_size1),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_size2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size2, 1)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)

In [18]:
def predict_proba(model: nn.Module, X: np.ndarray, preprocessor: ColumnTransformer, device: torch.device, batch_size: int = 256) -> np.ndarray:
    X_processed = preprocessor.transform(X)
    pred_dataset = torch.utils.data.TensorDataset(torch.tensor(X_processed, dtype=torch.float32))
    pred_loader = DataLoader(pred_dataset, batch_size=batch_size, shuffle=False)
    pred_pbar = tqdm(pred_loader, desc="Predicting")

    model.eval()
    all_probabilities = []
    with torch.no_grad():
        for batch in pred_pbar:
            inputs = batch[0].to(device)
            logits = model(inputs)
            probabilities = torch.sigmoid(logits)
            all_probabilities.append(probabilities.cpu().numpy())

    all_probabilities = np.concatenate(all_probabilities, axis=0)
    return all_probabilities.squeeze()

In [19]:
load_path = 'models_joblib/tender_mlp_model_and_preprocessor.pth'
checkpoint = torch.load(load_path, map_location=device, weights_only=False)
loaded_preprocessor = checkpoint['preprocessor']
loaded_input_size = checkpoint['input_size']
model_state_dict = checkpoint['model_state_dict']
loaded_model = TenderNet(input_size=loaded_input_size).to(device)
loaded_model.load_state_dict(model_state_dict)
loaded_model.eval()
predictions_mlp = predict_proba(loaded_model, prediction_data_for_model, loaded_preprocessor, device)

Predicting: 100%|██████████| 1/1 [00:00<00:00, 37.20it/s]


In [20]:
results_df = pd.DataFrame({
                'ИНН поставщика': supplier_inns_predict,
                'SVM_prediction': predictions_svm,
                'KNN_prediction': predictions_knn,
                'MLP_prediction': predictions_mlp
            })

            
results_df_sorted_svm = results_df.sort_values(by='SVM_prediction', ascending=False)
print("\nРезультаты предсказаний (Топ 5 по SVM):")
print(results_df_sorted_svm.head())


Результаты предсказаний (Топ 5 по SVM):
    ИНН поставщика  SVM_prediction  KNN_prediction  MLP_prediction
31      7707049388        0.999992        0.950025        0.981833
19      3818015926        0.960350        0.863686        0.980807
37    260702329791        0.957683        0.898636        0.980656
40    744913107404        0.956235        0.907989        0.980615
7       3801125310        0.955629        0.895745        0.980579


In [21]:
results_df_sorted_knn = results_df.sort_values(by='KNN_prediction', ascending=False)
print("\nРезультаты предсказаний (Топ 5 по KNN):")
print(results_df_sorted_knn.head())


Результаты предсказаний (Топ 5 по KNN):
    ИНН поставщика  SVM_prediction  KNN_prediction  MLP_prediction
35      7804513164        0.955423        0.951677        0.980382
31      7707049388        0.999992        0.950025        0.981833
27      6683004810        0.819413        0.947895        0.946661
40    744913107404        0.956235        0.907989        0.980615
37    260702329791        0.957683        0.898636        0.980656


In [22]:
results_df_sorted_mlp = results_df.sort_values(by='MLP_prediction', ascending=False)
print("\nРезультаты предсказаний (Топ 10 по MLP):")
print(results_df_sorted_mlp.head(10))


Результаты предсказаний (Топ 10 по MLP):
    ИНН поставщика  SVM_prediction  KNN_prediction  MLP_prediction
31      7707049388        0.999992        0.950025        0.981833
19      3818015926        0.960350        0.863686        0.980807
34      7801450808        0.952840        0.893952        0.980685
37    260702329791        0.957683        0.898636        0.980656
9       3808092294        0.955465        0.896332        0.980641
13      3811173994        0.954246        0.898214        0.980616
5       2540130907        0.954246        0.898214        0.980616
36      8506002215        0.954246        0.898214        0.980616
40    744913107404        0.956235        0.907989        0.980615
7       3801125310        0.955629        0.895745        0.980579


In [23]:
tender_subset_df[(tender_subset_df['Реестровый номер публикации'] == 32110673060) & (tender_subset_df['Победитель'] == 1)]['ИНН поставщика']

21795    744913107404
Name: ИНН поставщика, dtype: int64

KNN и SVM ранжируют победителя 4м, в то время как MLP определила его 9м, но глобально результаты хорошие - всего был 41 поставщик для предсказания. Проверим среднее попадание по части валидных (на данный момент) тендеров

In [24]:
import random

all_known_suppliers= set(data['ИНН поставщика'].unique())

def get_supplier_stats(inn: str):
    supplier_records = data[data['ИНН поставщика'] == inn]
    wins = 0
    if 'Победитель' in supplier_records.columns:
        wins = int(supplier_records['Победитель'].sum())
    return {
            'records_count': len(supplier_records),
            'wins_count': wins
    }

valid_tenders = []
all_tender_ids = data['Реестровый номер публикации'].unique()

processed_count = 0
for tender_id in all_tender_ids:
    tender_data = data[data['Реестровый номер публикации'] == tender_id]
    participants = tender_data['ИНН поставщика'].unique()
    num_participants = len(participants)
    if all(inn in all_known_suppliers for inn in participants) and num_participants >= 1:
        winner_data = tender_data[tender_data['Победитель'] == 1]
        if not winner_data.empty:
            winner_inn = winner_data['ИНН поставщика'].iloc[0]
            valid_tenders.append({
                        'tender_id': tender_id,
                        'participants_count': num_participants,
                        'winner_inn': winner_inn
                    })
    processed_count += 1

print(f"\nНайдено {len(valid_tenders)} валидных тендеров")
selected_tenders_data = []
selected_sample = random.sample(valid_tenders, 150)


Найдено 54605 валидных тендеров


In [25]:
from contextlib import redirect_stdout

winner_ranks_list = []
output_filename = 'check_results.txt'

with open(output_filename, 'w', encoding='utf-8') as f:
    with redirect_stdout(f):
        for i in selected_sample:
            if not np.isnan(i['winner_inn']):
                tender = i['tender_id']
                tender_subset_df = data[data['Реестровый номер публикации'] == tender]
                winner_series = tender_subset_df[tender_subset_df['Победитель'] == 1]['ИНН поставщика']
                print(f"Реальный победитель: {winner_series}")
                new_tender_info = format_tender_info_dict(tender_subset_df)
                new_tender_df = pd.DataFrame([new_tender_info])

                use_filtered = True 
                active_supplier_pool = filter_suppliers(supplier_historical_data, new_tender_df)
                num_suppliers_to_predict = active_supplier_pool.shape[0]

                if num_suppliers_to_predict > 0:
                    tender_df_repeated = pd.concat([new_tender_df] * num_suppliers_to_predict, ignore_index=True)
                    suppliers_reset = active_supplier_pool.reset_index(drop=True)
                    prediction_input_df = pd.concat([suppliers_reset, tender_df_repeated], axis=1)
                    all_features_needed = num_features + cat_features + passthrough_features
                    final_columns = ['ИНН поставщика'] + all_features_needed
                    prediction_input_final = prediction_input_df[final_columns].copy()
                    supplier_inns_predict = prediction_input_final['ИНН поставщика']
                    prediction_data_for_model = prediction_input_final.drop(columns=['ИНН поставщика'])

                    predictions_knn = loaded_knn_pipeline.predict_proba(prediction_data_for_model)[:, 1]
                    predictions_svm = loaded_svm_pipeline.predict_proba(prediction_data_for_model)[:, 1]
                    predictions_mlp = predict_proba(loaded_model, prediction_data_for_model, loaded_preprocessor, device)

                    results_df = pd.DataFrame({
                    'ИНН поставщика': supplier_inns_predict,
                    'SVM_prediction': predictions_svm,
                    'KNN_prediction': predictions_knn,
                    'MLP_prediction': predictions_mlp
                    })

                    ranks = {
                    'tender_id': tender,
                    'real_winners_inns': winner_series,
                    'num_participants_predicted': num_suppliers_to_predict
                    }

                    if len(winner_series) > 0: 
                        predicted_inns_set = set(results_df['ИНН поставщика'].values)
                        winners_found_in_predictions = [inn for inn in winner_series if inn in predicted_inns_set]

                        if not winners_found_in_predictions:
                            print(f"  Предупреждение: Ни один из реальных победителей {winner_series} не найден в списке предсказанных поставщиков ({num_suppliers_to_predict} шт.).")
                            continue
                        else:
                            if len(winners_found_in_predictions) < len(winner_series):
                                print(f"  Предупреждение: Не все реальные победители найдены в предсказаниях. Найдены: {winners_found_in_predictions}")

                            for model_name in ['SVM', 'KNN', 'MLP']:
                                col_name = f'{model_name}_prediction'
                                sorted_df = results_df.sort_values(by=col_name, ascending=False).reset_index(drop=True)
                                found_ranks_for_model = []
                                for winner_inn in winners_found_in_predictions:
                                    winner_position = sorted_df[sorted_df['ИНН поставщика'] == winner_inn].index
                                    if not winner_position.empty:
                                        rank = winner_position[0] + 1 
                                        found_ranks_for_model.append(rank)

                                if found_ranks_for_model:
                                    avg_rank = np.mean(found_ranks_for_model)
                                    ranks[f'{model_name}_rank'] = avg_rank
                                    print(f"  Ранги найденных победителей ({model_name}): {found_ranks_for_model} -> Средний ранг: {avg_rank:.2f} из {num_suppliers_to_predict}")
                                    winner_ranks_list.append(ranks)
                                    print("-" * 70) 
                    else:
                        continue
                else: 
                    continue
            else:
                continue

Predicting: 100%|██████████| 1/1 [00:00<00:00, 663.13it/s]


In [26]:
winner_ranks_df = pd.DataFrame(winner_ranks_list)
winner_ranks_df.tail(10)

,tender_id,real_winners_inns,num_participants_predicted,SVM_rank,KNN_rank,MLP_rank
407,32110207993,"62932 7714731464 Name: ИНН поставщика, dtyp...",3,2.0,3.0,3.0
408,32211220404,"57512 7710425868 Name: ИНН поставщика, dtyp...",1,1.0,1.0,1.0
409,32211220404,"57512 7710425868 Name: ИНН поставщика, dtyp...",1,1.0,1.0,1.0
410,32211220404,"57512 7710425868 Name: ИНН поставщика, dtyp...",1,1.0,1.0,1.0
411,32110666361,"91140 9715253140 Name: ИНН поставщика, dtyp...",78,9.0,5.0,2.0
412,32110666361,"91140 9715253140 Name: ИНН поставщика, dtyp...",78,9.0,5.0,2.0
413,32110666361,"91140 9715253140 Name: ИНН поставщика, dtyp...",78,9.0,5.0,2.0
414,32110059136,"42385 2901264605 Name: ИНН поставщика, dtyp...",20,12.0,3.0,12.0
415,32110059136,"42385 2901264605 Name: ИНН поставщика, dtyp...",20,12.0,3.0,12.0
416,32110059136,"42385 2901264605 Name: ИНН поставщика, dtyp...",20,12.0,3.0,12.0


In [27]:
winner_ranks_df[['SVM_rank', 'KNN_rank', 'MLP_rank']].agg(['mean', 'median', 'std'])

,SVM_rank,KNN_rank,MLP_rank
mean,12.959233,11.635492,13.736211
median,3.000000,3.000000,3.000000
std,31.101866,24.978737,28.667369


In [28]:
#код для того, чтобы посмотреть на конкретные тендеры

tenders_to_check = [32211357812]

for tender in tenders_to_check:
    tender_subset_df = data[data['Реестровый номер публикации'] == tender]
    new_tender_info = format_tender_info_dict(tender_subset_df)
    new_tender_df = pd.DataFrame([new_tender_info])
    use_filtered = True 
    if 'regions_set' in supplier_historical_data.columns and 'spheres_set' in supplier_historical_data.columns:
        active_supplier_pool = filter_suppliers(supplier_historical_data, new_tender_df)

    num_suppliers_to_predict = active_supplier_pool.shape[0]
    if num_suppliers_to_predict > 0:
        tender_df_repeated = pd.concat([new_tender_df] * num_suppliers_to_predict, ignore_index=True)
        suppliers_reset = active_supplier_pool.reset_index(drop=True)
        prediction_input_df = pd.concat([suppliers_reset, tender_df_repeated], axis=1)
        all_features_needed = num_features + cat_features + passthrough_features
        final_columns = ['ИНН поставщика'] + all_features_needed
        prediction_input_final = prediction_input_df[final_columns].copy() 
        # Сохраняем ИНН поставщиков отдельно для сопоставления результатов
        supplier_inns_predict = prediction_input_final['ИНН поставщика']
        prediction_data_for_model = prediction_input_final.drop(columns=['ИНН поставщика'])

        predictions_knn = loaded_knn_pipeline.predict_proba(prediction_data_for_model)[:, 1]
        predictions_svm = loaded_svm_pipeline.predict_proba(prediction_data_for_model)[:, 1]
        predictions_mlp = predict_proba(loaded_model, prediction_data_for_model, loaded_preprocessor, device)
        results_df = pd.DataFrame({
                'ИНН поставщика': supplier_inns_predict,
                'SVM_prediction': predictions_svm,
                'KNN_prediction': predictions_knn,
                'MLP_prediction': predictions_mlp
            })

        print(f"Реальный победитель тендера: {tender_subset_df[(tender_subset_df['Реестровый номер публикации'] == tender) & (tender_subset_df['Победитель'] == 1)]['ИНН поставщика']}\n")

        results_df_sorted_svm = results_df.sort_values(by='SVM_prediction', ascending=False)
        print("\nРезультаты предсказаний (Топ 10 по SVM):")
        print(results_df_sorted_svm.head(10))
        results_df_sorted_knn = results_df.sort_values(by='KNN_prediction', ascending=False)
        print("\nРезультаты предсказаний (Топ 10 по KNN):")
        print(results_df_sorted_knn.head(10))
        results_df_sorted_mlp = results_df.sort_values(by='MLP_prediction', ascending=False)
        print("\nРезультаты предсказаний (Топ 10 по MLP):")
        print(results_df_sorted_mlp.head(10))

Получено 79 поставщиков по региону 'Краснодарский край' и сфере '[ОКПД2 71.12] Услуги в области инженерно-технического проектирования и связанные технические консультативные услуги, [ОКВЭД2 71.12] Деятельность в области инженерных изысканий, инженерно-технического проектирования, управления проектами строительства, выполнения строительного контроля и авторского надзора, предоставление технических консультаций в этих областях'


Predicting: 100%|██████████| 1/1 [00:00<00:00, 737.40it/s]

Реальный победитель тендера: 96245    2308131521
Name: ИНН поставщика, dtype: int64


Результаты предсказаний (Топ 10 по SVM):
    ИНН поставщика  SVM_prediction  KNN_prediction  MLP_prediction
50      7707049388        0.999987        0.958844        0.983238
48      7703803572        0.950484        1.000000        0.982225
20      2312270126        0.949250        0.892121        0.982125
68     70601173411        0.948931        0.807479        0.982064
8       2308188951        0.948928        0.907941        0.981978
49      7704060922        0.948796        0.849594        0.982071
5       2308110433        0.948309        0.893741        0.982024
61      7805721992        0.948217        1.000000        0.982299
67     10602494280        0.947992        0.893178        0.982019
23      2315136224        0.947725        0.892602        0.981961

Результаты предсказаний (Топ 10 по KNN):
    ИНН поставщика  SVM_prediction  KNN_prediction  MLP_prediction
19      2311287920        0